<a href="https://colab.research.google.com/github/Jungyeun-25/21_study/blob/master/CNN_basicMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, Dropout
from tensorflow.keras import Model

In [ ]:
from keras.datasets import cifar100
# cifar 데이터 불러오기
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [ ]:
x_train = tf.image.convert_image_dtype(x_train, tf.float32)
x_test = tf.image.convert_image_dtype(x_test, tf.float32)

In [ ]:
# mnist = tf.keras.datasets.mnist

# (x_train,y_train),(x_test,y_test) = mnist.load_data()
# x_train,x_test = x_train/255.0,x_test/255.0

# x_train = x_train.reshape(-1,28,28,1) # {train data의 개수}*28*28 -> {train data의 개수}*28*28*1 
# x_test = x_test.reshape(-1,28,28,1)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train,y_train)).shuffle(buffer_size=10000).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [ ]:
train_dataset

<BatchDataset shapes: ((None, 32, 32, 3), (None, 1)), types: (tf.float32, tf.int64)>

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel,self).__init__()
    self.conv1 = Conv2D(filters=32,kernel_size=3,activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(units=128,activation='relu')
    self.d2 = Dense(units=100,activation='softmax')

  def call(self,x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [ ]:
lr = 0.01
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer =  tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')



In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
  

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
import time

EPOCHS = 5

start = time.time()
for epoch in range(EPOCHS):
  for images,labels in train_dataset :
    train_step(images,labels)
  
  for test_images,test_labels in test_dataset:
    test_step(test_images,test_labels)
  
  template = 'EPOCH : {}, LOSS : {:.5f}, ACCURACY : {:.2f}%, test LOSS : {:.5f}, test ACCURACY : {:.2f}%'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
  
print("time :", time.time() - start)

EPOCH : 1, LOSS : 4.62800, ACCURACY : 1.01%, test LOSS : 4.60845, test ACCURACY : 1.00%
EPOCH : 2, LOSS : 4.61925, ACCURACY : 0.95%, test LOSS : 4.60875, test ACCURACY : 1.00%
EPOCH : 3, LOSS : 4.61643, ACCURACY : 0.93%, test LOSS : 4.60864, test ACCURACY : 1.00%
EPOCH : 4, LOSS : 4.61498, ACCURACY : 0.92%, test LOSS : 4.60868, test ACCURACY : 1.00%
EPOCH : 5, LOSS : 4.61409, ACCURACY : 0.93%, test LOSS : 4.60873, test ACCURACY : 1.00%
time : 23.76604437828064
